# Демо-скрипт для выравнивания параллельных текстов на армянском и русском языках

lingtrain-aligner==1.0.2

## Сетап

В качестве инпута подаются два txt-файла на армянском и русском. В скрипте предполагается, что эти файлы лежат на уровне скрипта. Если они лежат в другом месте, нужно прописать путь до файла в переменные `am_input` и `ru_input`. Название для файла, получаемого на выходе (html-документ с раскрашенными предложениями и выровненный csv) нужно задать в переменной `project_name`.

В идеале хорошо бы разметить файлы на предмет метаинформации: `Лю Ци Синь%%%%%author.`, `Задача трёх тел%%%%%title.` и т.д., иначе заголовок сливается с основным текстом, т.к. между ними нет разделяющего знака препинания. Теги для такой разметки можно посмотреть в "полезных ссылках ниже".

Результаты обработки сохраняются в папку `result` по умолчанию, но можно поменять название папки установив его в переменной `output_dir`.

## Полезные ссылки
- Статья про инструмент и метки для указания метинформации https://habr.com/ru/articles/704958/
- github проекта https://github.com/averkij/lingtrain-aligner/tree/main
- основывалась на скрипте https://colab.research.google.com/drive/1lgmgCJuFAqjEI2zqn9RWPcQuO6rxC80f?usp=sharing#scrollTo=bZ0ZlbNqqjV6

In [ ]:
#@title Установим зависимости

!pip install -q -U lingtrain-aligner==1.0.2
!pip install -q razdel dateparser sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 847.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00


In [ ]:
#@title Импортируем библиотеки
import os
import zipfile
import shutil
import datetime

from lingtrain_aligner import preprocessor, splitter, aligner, resolver, reader, helper, vis_helper
import pandas as pd
import itertools

In [ ]:
# зададим общие переменные
output_dir = 'result'
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

models = ["sentence_transformer_multilingual", "sentence_transformer_multilingual_labse"]
model_name = models[0]

In [ ]:
#@title Класс для обработки (нужно запустить)
class pair2Align:
    def __init__(self, text_from_name, text_to_name, project_name, lang_from, lang_to, model):

      self.text_from_name = text_from_name
      self.text_to_name = text_to_name
      assert project_name, 'Нужно передать название текста!'
      self.project_name = project_name
      self.db_path = project_name + '.db'

      self.lang_from = lang_from
      self.lang_to = lang_to

      # конфигурации для формирования html-документа
      self.lang_ordered = ["from", "to"]

      # запускаем пайп
      self.load_from()
      self.load_to()
      self.split_by_sent()
      self.conflicts = self.align()
      self.resolve_conflicts()
      self.get_aligned()

    def load_from(self):
    # Оригинальный текст
      with open(self.text_from_name, "r", encoding="utf8") as input1:
        self.text_from = input1.readlines()
      print('Загружен оригинальный текст')

    def load_to(self):
    # Текст перевода
      with open(self.text_to_name, "r", encoding="utf8") as input2:
        self.text_to = input2.readlines()
      print('Загружен текст перевода')

    def split_by_sent(self):
      # Добавим метки абзацев, они пригодятся позже
      text_from_prepared = preprocessor.mark_paragraphs(self.text_from)
      text_to_prepared = preprocessor.mark_paragraphs(self.text_to)

      self.splitted_from = splitter.split_by_sentences_wrapper(text_from_prepared, lang_from)
      self.splitted_to = splitter.split_by_sentences_wrapper(text_to_prepared, lang_to)

      print('Предложений в оригинальном тексте:', len(self.splitted_from))
      print('Предложений в тексте перевода:', len(self.splitted_to))

    def align(self):
      """
      for c in conflicts_to_solve:
        resolver.show_conflict(db_path, c)
      """

      if os.path.isfile(self.db_path):
        os.unlink(self.db_path)

      aligner.fill_db(self.db_path, lang_from, lang_to, self.splitted_from, self.splitted_to)

      batch_ids = [0,1]

      aligner.align_db(self.db_path, \
                      model_name, \
                      batch_size=100, \
                      window=40, \
                      batch_ids=batch_ids, \
                      save_pic=False,
                      embed_batch_size=10, \
                      normalize_embeddings=True, \
                      show_progress_bar=True
                      )
      vis_helper.visualize_alignment_by_db(self.db_path, output_path='viz.png', lang_name_from=self.lang_from, lang_name_to=self.lang_to, batch_size=400, size=(800,800), plt_show=True)

      return resolver.get_all_conflicts(self.db_path, min_chain_length=2, max_conflicts_len=6, batch_id=-1) # conflicts_to_solve, rest -> resolver.get_statistics(conflicts_to_solve), resolver.get_statistics(rest)


    def resolve_conflicts(self, steps=5):
      batch_id = -1 #выровнять все доступные батчи

      for i in range(steps):
          conflicts, rest = resolver.get_all_conflicts(self.db_path, min_chain_length=2+i, max_conflicts_len=5*(i+1), batch_id=batch_id, handle_start=True, handle_finish=True)
          resolver.resolve_all_conflicts(self.db_path, conflicts, model_name, show_logs=False)
          vis_helper.visualize_alignment_by_db(self.db_path, output_path='viz.png', lang_name_from=self.lang_from, lang_name_to=self.lang_to, batch_size=400, size=(600,600), plt_show=True)

          if len(rest) == 0: break
      print('Конфликты разрешены')

    def save_book(self, output_filename, output_dir=None, custom_styles=[]):
      """
      # можно передать свои правила для оформления книжки
      my_style = [
          '{"background": "#A2E4B8", "color": "black", "border-bottom": "0px solid red"}',
          '{"background": "#FFC1CC", "color": "black"}',
          '{"background": "#9BD3DD", "color": "black"}',
          '{"background": "#FFFCC9", "color": "black"}'
          ]
      save_aligned(custom_styles=my_style)
      """
      output_path = os.path.join(output_dir, output_filename +".html")
      if not os.path.exists(output_dir):
        os.mkdir(output_dir)

      paragraphs, delimeters, metas, sent_counter = reader.get_paragraphs(
          self.db_path, direction="to"
      )

      reader.create_book(
          lang_ordered=self.lang_ordered,
          paragraphs=paragraphs,
          delimeters=delimeters,
          metas=metas,
          sent_counter=sent_counter,
          output_path=output_path,
          template="pastel_fill",
          styles=custom_styles,
      )

    def get_aligned(self):

      paragraphs_from, paragraphs_to, meta, _ = reader.get_paragraphs(self.db_path)

      sents_from = list(itertools.chain.from_iterable(paragraphs_from['from']))
      sents_to = list(itertools.chain.from_iterable(paragraphs_from['to']))

      self.aligned_df = pd.DataFrame(data=[sents_from, sents_to], index=[self.lang_from, self.lang_to]).T

    @staticmethod
    def save_table(df, output_dir, out_filename):
      """
      pair2Align.save_table(proj.aligned_df, output_dir, out_filename=project_name)
      """
      # сохраняем в excel а не в csv на случай, если нужно будет подправить что-то руками
      df.to_excel(f'{output_dir}/{out_filename}.xlsx', index=False)


## Эту часть нужно перезапускать для каждой пары текстов

Если текстов много (например, новостных), нужно добавить в скрипт цикл, который бы определял пары одного и того же текста (должны быть названы по маске) и автоматически запускал функцию. Но в таком случае уже не получится отслеживать качество выравнивания -- получается такой tradeoff.

In [ ]:
#@title Переменные, уникальные для каждой пары текстов

# название книги
project_name = "rooster" #@param {"type":"string"}

cur_timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
db_path = f"{cur_timestamp}.db"

# название файла на русском (до .txt)
ru_input = f"tumanyan/ru_{project_name}" #@param {"type":"string"}
# название файла на армянском (до .txt)
am_input = f"tumanyan/am_{project_name}" #@param {"type":"string"}

ru_input = ru_input+'.txt'
am_input = am_input+'.txt'

# язык оригинала
lang_from = "hy" # @param ["hy","ru"]
# язык перевода
lang_to = "ru" # @param ["hy","ru"]

In [ ]:
proj = pair2Align(text_from_name=am_input, text_to_name=ru_input, project_name=project_name, lang_from=lang_from, lang_to=lang_to, model=model_name)

In [ ]:
proj.aligned_df

In [ ]:
# сохраняем таблицу с выровненными предложениями в папку
pair2Align.save_table(proj.aligned_df, output_dir, out_filename=project_name)

In [ ]:
# если хотим посмотреть на выровненную книжку, сохраняем ее в папку books
# ячейка закоменчена чтобы случайно не запустилась
# proj.save_book(output_filename=project_name, output_dir='books')

## После того, как мы закончили обрабатывать файлы, можно сохранить папку results в архив и скачать

ячейка закоменчена чтобы случайно не запустилась

In [ ]:
# zip_filename = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
# shutil.make_archive(zip_filename, 'zip', output_dir)

# from google.colab import files

# files.download(zip_filename+'.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>